In [ ]:
# Import the libraries

import os
import matplotlib.pyplot as plt
import numpy as np
from skimage import measure, io, img_as_ubyte
from skimage.color import rgb2gray, rgba2rgb
from skimage.filters import threshold_otsu

In [ ]:
class WesternBlotQuantification():
    
    CONTROL = {}
    EXPERIMENTAL = {}
    
    def __init__(self, conc, blot_img, control = False):
        self.conc = conc
        self.blot_img = blot_img
        self.control = control
        
    def read_gray(self):
        return img_as_ubyte(rgb2gray(rgba2rgb(io.imread(self.blot_img))))
    
    def show_gray(self):
        plt.imshow(self.read_gray())
        plt.show()